# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [402]:
# Import libraries
import numpy as np
import pandas as pd

In [442]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
print np.sum(student_data.passed == 'yes')
print np.sum(student_data.passed == 'no')
print np.count_nonzero(student_data.passed == 'yes')

# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!
265
130
265


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [432]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = np.size(student_data, 0)
n_features = np.size(student_data, 1)
n_passed =np.sum(student_data.passed == 'yes')
n_failed =np.sum(student_data.passed == 'no')
grad_rate = float(n_passed)/(n_students)
print float(n_passed)/ float(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate*100)


0.670886075949
Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 67.09%


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes
5,GP,M,16,U,LE3,T,4,3,services,other,...,yes,no,5,4,2,1,2,5,10,yes
6,GP,M,16,U,LE3,T,2,2,other,other,...,yes,no,4,4,4,1,1,3,0,yes
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,no,no,4,1,4,1,1,1,6,no
8,GP,M,15,U,LE3,A,3,2,services,other,...,yes,no,4,2,2,1,1,1,0,yes
9,GP,M,15,U,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,yes


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [441]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [406]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))


Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [407]:
#y_all = preprocess_features(y_all)
#print y_all #= y_all.replace(['yes', 'no'], [1, 0])
#print "Processed feature columns ({}):-\n{}".format(len(y_all.columns), list(y_all.columns))

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [440]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

from sklearn.cross_validation import train_test_split
# Shuffle Dataframe
#X_all_shuf = X_all.reindex(np.random.permutation(X_all.index))
#Y_all_shuf = y_all.reindex(np.random.permutation(y_all.index))

#X_train = X_all_shuf[:num_train]
#y_train = Y_all_shuf[:num_train]
#X_test = X_all_shuf[num_train+1:num_all]
#y_test = Y_all_shuf[num_train+1:num_all]
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = float(num_test)/num_all, random_state = 42 )
print y_train
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data
y_test

210     no
75     yes
104    yes
374    yes
16     yes
66     yes
211    yes
7       no
19     yes
301    yes
350     no
365    yes
118     no
321     no
296     no
381     no
373     no
60     yes
79      no
281    yes
245    yes
250     no
218     no
109    yes
17     yes
165    yes
24      no
126    yes
346    yes
176    yes
      ... 
344    yes
235    yes
252     no
21     yes
313    yes
160     no
276     no
191     no
293    yes
343     no
257    yes
308    yes
149    yes
130     no
151    yes
359    yes
99      no
372    yes
87     yes
330     no
214    yes
121    yes
394     no
20     yes
188     no
71     yes
106     no
270     no
348    yes
102    yes
Name: passed, dtype: object
Training set: 300 samples
Test set: 95 samples


78     yes
371    yes
248     no
55     yes
390     no
223    yes
42     yes
234     no
316     no
116    yes
9      yes
209     no
323    yes
393    yes
258    yes
155     no
124     no
272    yes
324    yes
334     no
15     yes
391    yes
167    yes
0       no
140     no
113    yes
77     yes
33     yes
101    yes
76     yes
      ... 
182    yes
255     no
358    yes
317     no
5      yes
45      no
84     yes
289    yes
110    yes
364    yes
82      no
114     no
3      yes
18      no
181    yes
194    yes
173     no
63      no
253     no
225     no
331    yes
383     no
227    yes
342    yes
131     no
320    yes
175     no
238    yes
368    yes
367     no
Name: passed, dtype: object

## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [409]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
#clf = SVC()
#from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
#clf = LinearRegression()# MLPClassifier(algorithm='l-bfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1) #SVC()
from sklearn.naive_bayes import GaussianNB
#clf = GaussianNB()
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training GradientBoostingClassifier...
Done!
Training time (secs): 0.146
GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [410]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')
    #return f1_score(target.values, y_pred)

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)


Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.973871733967


In [411]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.779411764706


In [412]:
float(num_test)/num_all

0.24050632911392406

In [413]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [414]:
# TODO: Train and predict using two other models

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [415]:
# TODO: Fine-tune your model and report the best F1 score

In [416]:
from sklearn import grid_search
from sklearn.metrics import make_scorer

In [434]:
#parameters = {'max_depth':(1,2,3,4,5,6,7,8,9,10)}
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 2, 5, 7, 10]}
parameters = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth':(1,2,3,4,5,6,7,8,9,10),
              'min_samples_leaf': [3, 5, 9, 17],
              }
f1_scorer=make_scorer(f1_score, pos_label="yes")
reg = grid_search.GridSearchCV( GradientBoostingClassifier(), parameters, verbose=10, scoring = f1_scorer)
train_classifier(reg, X_train, y_train)


Training GridSearchCV...
Fitting 3 folds for each of 160 candidates, totalling 480 fits
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=3 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=3, score=0.842767 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=3 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=3, score=0.807692 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=3 ..............


[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.3s


[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=3, score=0.781457 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=5 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=5, score=0.835443 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=5 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=5, score=0.807692 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=5 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=5, score=0.781457 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=9 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=9, score=0.835443 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=9 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=9, score=0.810127 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=9 ..............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=9, score=0.781457 -   0.1s


[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.4s


[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=17, score=0.820513 -   0.1s
[CV] learning_rate=0.1, max_depth=1, min_samples_leaf=17 .............
[CV]  learning_rate=0.1, max_depth=1, min_samples_leaf=17, score=0.750000 -   0.1s
[CV] learning_rate=0.1, max_depth=2, min_samples_leaf=3 ..............
[CV]  learning_rate=0.1, max_depth=2, min_samples_leaf=3, score=0.830189 -   0.1s
[CV] learning_rate=0.1, max_depth=2, min_samples_leaf=3 ..............


...

[Parallel(n_jobs=1)]: Done 420 tasks       | elapsed:   54.5s


[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=3, score=0.807692 -   0.2s
[CV] learning_rate=0.01, max_depth=6, min_samples_leaf=3 .............
[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=3, score=0.807692 -   0.2s
[CV] learning_rate=0.01, max_depth=6, min_samples_leaf=3 .............
[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=3, score=0.802548 -   0.2s
[CV] learning_rate=0.01, max_depth=6, min_samples_leaf=5 .............
[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=5, score=0.810127 -   0.2s
[CV] learning_rate=0.01, max_depth=6, min_samples_leaf=5 .............
[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=5, score=0.802548 -   0.1s
[CV] learning_rate=0.01, max_depth=6, min_samples_leaf=5 .............
[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=5, score=0.794702 -   0.2s
[CV] learning_rate=0.01, max_depth=6, min_samples_leaf=9 .............
[CV]  learning_rate=0.01, max_depth=6, min_samples_leaf=9, score=0.815287 - 

[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   59.2s


[CV]  learning_rate=0.01, max_depth=8, min_samples_leaf=9, score=0.815287 -   0.2s
[CV] learning_rate=0.01, max_depth=8, min_samples_leaf=9 .............
[CV]  learning_rate=0.01, max_depth=8, min_samples_leaf=9, score=0.802548 -   0.1s
[CV] learning_rate=0.01, max_depth=8, min_samples_leaf=9 .............
[CV]  learning_rate=0.01, max_depth=8, min_samples_leaf=9, score=0.800000 -   0.1s
[CV] learning_rate=0.01, max_depth=8, min_samples_leaf=17 ............
[CV]  learning_rate=0.01, max_depth=8, min_samples_leaf=17, score=0.802469 -   0.1s
[CV] learning_rate=0.01, max_depth=8, min_samples_leaf=17 ............
[CV]  learning_rate=0.01, max_depth=8, min_samples_leaf=17, score=0.810127 -   0.1s
[CV] learning_rate=0.01, max_depth=8, min_samples_leaf=17 ............
[CV]  learning_rate=0.01, max_depth=8, min_samples_leaf=17, score=0.817610 -   0.1s
[CV] learning_rate=0.01, max_depth=9, min_samples_leaf=3 .............
[CV]  learning_rate=0.01, max_depth=9, min_samples_leaf=3, score=0.807692

[Parallel(n_jobs=1)]: Done 480 tasks       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  1.1min finished


In [436]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)



Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.973871733967


In [438]:

train_f1_score = predict_labels(reg, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.838174273859


In [439]:
reg.best_estimator_

GradientBoostingClassifier(init=None, learning_rate=0.02, loss='deviance',
              max_depth=1, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=3, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [421]:
reg.best_params_

{'C': 1, 'kernel': 'rbf'}

In [427]:

train_f1_score = predict_labels(reg, X_test, y_test)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.783783783784
